In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

VBox()

Starting Spark application


Exception in thread cell_monitor-1:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1742363616088_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Initialize Spark session
spark = SparkSession.builder.appName("PokerHandClassification").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Load the dataset into a Spark DataFrame
file_path = "s3://poker-hand-project/test/poker.csv"
poker_spark_df = spark.read.csv(file_path, header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
print(f"Number of rows in poker hand dataset: {poker_spark_df.count()}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of rows in poker hand dataset: 1000000

In [5]:
# Display the first few rows
poker_spark_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---+---+---+---+---+---+---+---+---+-----+
| S1| C1| S2| C2| S3| C3| S4| C4| S5| C5|CLASS|
+---+---+---+---+---+---+---+---+---+---+-----+
|  1|  1|  1| 13|  2|  4|  2|  3|  1| 12|    0|
|  3| 12|  3|  2|  3| 11|  4|  5|  2|  5|    1|
|  1|  9|  4|  6|  1|  4|  3|  2|  3|  9|    1|
|  1|  4|  3| 13|  2| 13|  2|  1|  3|  6|    1|
|  3| 10|  2|  7|  1|  2|  2| 11|  4|  9|    0|
+---+---+---+---+---+---+---+---+---+---+-----+
only showing top 5 rows

In [6]:
# Check for null values
null_counts = poker_spark_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in poker_spark_df.columns])
null_counts.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---+---+---+---+---+---+---+---+---+-----+
| S1| C1| S2| C2| S3| C3| S4| C4| S5| C5|CLASS|
+---+---+---+---+---+---+---+---+---+---+-----+
|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|    0|
+---+---+---+---+---+---+---+---+---+---+-----+

In [7]:
# Assemble feature columns into a single vector
feature_cols = [col for col in poker_spark_df.columns if col != "CLASS"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
poker_spark_df = assembler.transform(poker_spark_df)
poker_spark_df = poker_spark_df.select("features", col("CLASS").alias("label"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Split dataset into training and testing sets
train_df, test_df = poker_spark_df.randomSplit([0.8, 0.2], seed=42)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Train a Logistic Regression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)
lr_model = lr.fit(train_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Make predictions
predictions = lr_model.transform(test_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Evaluate model performance
evaluator = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Logistic Regression Accuracy: {accuracy:.4f}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Logistic Regression Accuracy: 0.5028

In [12]:
# Define parameter grid for hyperparameter tuning
param_grid = ParamGridBuilder().addGrid(lr.regParam, [0.01, 0.1, 1.0]).build()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Cross-validation setup
crossval = CrossValidator(estimator=lr, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)
cv_model = crossval.fit(train_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Evaluate best model from cross-validation
cv_predictions = cv_model.transform(test_df)
cv_accuracy = evaluator.evaluate(cv_predictions)
print(f"Tuned Logistic Regression Accuracy: {cv_accuracy:.4f}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Tuned Logistic Regression Accuracy: 0.5028

In [15]:
# Train a Random Forest Classifier
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100)
rf_model = rf.fit(train_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
param_grid = ParamGridBuilder().addGrid(rf.maxDepth, [2, 5, 10]).build()

# Cross-validation setup for Random Forest
rf_crossval = CrossValidator(estimator=rf, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)
rf_cv_model = rf_crossval.fit(train_df)

# Make predictions using the best Random Forest model
rf_cv_predictions = rf_cv_model.transform(test_df)
rf_cv_accuracy = evaluator.evaluate(rf_cv_predictions)
print(f"Tuned Random Forest Accuracy: {rf_cv_accuracy:.4f}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Tuned Random Forest Accuracy: 0.6268

In [ ]:
from pyspark.ml.classification import OneVsRest

gbt_classifier = GBTClassifier(labelCol="label", featuresCol="features", maxIter=50, maxDepth=5)  # Base GBT model
ovr = OneVsRest(classifier=gbt_classifier, labelCol="label")  # Wrap GBT in OneVsRest

# Cross-validation setup for OneVsRest with GBT
gbt_param_grid = ParamGridBuilder().addGrid(gbt_classifier.maxIter, [15, 50]).build()
gbt_crossval = CrossValidator(estimator=ovr, estimatorParamMaps=gbt_param_grid, evaluator=evaluator, numFolds=2, parallelism=4)

gbt_cv_model = gbt_crossval.fit(train_df)  # Train the OneVsRest model
gbt_cv_predictions = gbt_cv_model.transform(test_df)
gbt_cv_accuracy = evaluator.evaluate(gbt_cv_predictions)

print(f"Tuned OneVsRest (GBT) Accuracy: {gbt_cv_accuracy:.4f}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-17:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'
Exception in thread cell_monitor-16:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in

In [ ]:
from pyspark.ml.classification import OneVsRest

# Wrap LinearSVC in OneVsRest for multi-class classification
svc_classifier = LinearSVC(labelCol="label", featuresCol="features")
ovr = OneVsRest(classifier=svc_classifier, labelCol="label")

# Define parameter grid
svc_param_grid = ParamGridBuilder().addGrid(svc_classifier.maxIter, [10, 15]).addGrid(svc_classifier.regParam, [0.1, 0.01]).build()

# Cross-validation setup for OneVsRest with Linear SVC
svc_crossval = CrossValidator(estimator=ovr, estimatorParamMaps=svc_param_grid, evaluator=evaluator, numFolds=3)
svc_cv_model = svc_crossval.fit(train_df)

# Make predictions using the best Linear SVC model
svc_cv_predictions = svc_cv_model.transform(test_df)
svc_cv_accuracy = evaluator.evaluate(svc_cv_predictions)

print(f"Tuned OneVsRest (Linear SVC) Accuracy: {svc_cv_accuracy:.4f}")
